In [5]:
pip install gensim==4.3.2 --quiet

In [1]:
!pip install --force-reinstall "scipy==1.10.1" "numpy==1.26.4" "gensim==4.3.2" --quiet


ERROR: Ignored the following yanked versions: 1.11.0, 1.14.0rc1
ERROR: Ignored the following versions that require a different python version: 1.10.0 Requires-Python <3.12,>=3.8; 1.10.0rc1 Requires-Python <3.12,>=3.8; 1.10.0rc2 Requires-Python <3.12,>=3.8; 1.10.1 Requires-Python <3.12,>=3.8; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11; 1.9.0 Requires-Python >=3.8,<3.12; 1.9.0rc1 Requires-Python >=3.8,<3.12; 1.9.0rc2 Requires-Python >=3.8,<3.12; 1.9.0rc3 Requires-Python >=3.8,<3.12; 1.9.1 Requires-Python >=3.8,<3.12
ERROR: Could not find a version that satisfies the requirement scipy==1.10.1 (from versions:

In [2]:
# ==================================================
# 0. INSTALL & IMPORT LIBRARY
# ==================================================

# Fix for gensim/scipy incompatibility: ensure compatible scipy version is installed.
# This requires a runtime restart after execution for the changes to take full effect.
!pip install scipy==1.11.4 --quiet

import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from gensim.models import Word2Vec

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers


In [4]:
# ==================================================
# 1. BACA DATA
# ==================================================
# GANTI path & nama kolom sesuai dataset-mu
# Misal: kolom teks = 'tweet', label = 'label' (0 = non-sarkas, 1 = sarkas)

df = pd.read_csv("/content/cleaned_for_training_final (3).csv")

print("Contoh 5 data teratas:")
print(df.head())

print("\nDistribusi label:")
print(df['label'].value_counts())
print("\nPersentase label:")
print((df['label'].value_counts(normalize=True) * 100).round(2))


Contoh 5 data teratas:
                                           full_text  \
0   jangan terkecoh berita korup ingat #KaburAjaDulu   
1  Berharap Indonesia Emas tp pra-syarat menjadi ...   
2  wdyt #KaburAjaDulu ke kamboja jd admin judol l...   
3  Liat kakak gw tinggal di Belanda makmur2 aj hi...   
4  Culik aku kuliner sebelum berangkat lagi ke ne...   

                                          clean_text  label  
0  jangan kecoh berita korupsi ingat kabur saja dulu      0  
1  harap indonesia emas tapi pra syarat jadi indo...      1  
2  wdyt kabur saja dulu ke kamboja jadi admin jud...      0  
3  lihat kakak saya tinggal di belanda makmur saj...      0  
4  culik aku kuliner belum berangkat lagi ke nega...      1  

Distribusi label:
label
0    4374
1    2804
Name: count, dtype: int64

Persentase label:
label
0    60.94
1    39.06
Name: proportion, dtype: float64


In [6]:
# ==================================================
# SPLIT DATA 70:15:15
# ==================================================

# Train = 70%, sisanya 30%
X_train, X_temp, y_train, y_temp = train_test_split(
    df["clean_text"].values,
    df["label"].values,
    test_size=0.30,
    random_state=42,
    stratify=df["label"].values
)

# Dari sisa 30% → bagi menjadi val 15% & test 15%
# Artinya val = 0.5 * temp dan test = 0.5 * temp
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.50,
    random_state=42,
    stratify=y_temp
)

print("Jumlah train :", len(X_train), f"({(len(X_train)/len(df)*100):.2f} %)")
print("Jumlah val   :", len(X_val),   f"({(len(X_val)/len(df)*100):.2f} %)")
print("Jumlah test  :", len(X_test),  f"({(len(X_test)/len(df)*100):.2f} %)")


Jumlah train : 5024 (69.99 %)
Jumlah val   : 1077 (15.00 %)
Jumlah test  : 1077 (15.00 %)


In [8]:
from gensim.models.doc2vec import TaggedDocument

# Membuat TaggedDocument dari data TRAIN saja
train_tagged = [
    TaggedDocument(words=text.split(), tags=[f"train_{i}"])
    for i, text in enumerate(X_train)
]

len(train_tagged), train_tagged[0]

(5024,
 TaggedDocument(words=['besok', 'senin', 'kabur', 'saja', 'dulu'], tags=['train_0']))

In [9]:
VECTOR_SIZE = 300   # dimensi embedding dokumen
EPOCHS_D2V = 40

d2v_model = Doc2Vec(
    vector_size=VECTOR_SIZE,
    window=5,
    min_count=2,
    workers=4,
    dm=1,        # 1 = Distributed Memory (doc2vec DM), 0 = DBOW
    epochs=EPOCHS_D2V
)

d2v_model.build_vocab(train_tagged)
print("Jumlah kata di vocab:", len(d2v_model.wv))

d2v_model.train(
    train_tagged,
    total_examples=d2v_model.corpus_count,
    epochs=d2v_model.epochs
)


NameError: name 'Doc2Vec' is not defined

In [18]:
from gensim.models.doc2vec import Doc2Vec

VECTOR_SIZE = 300   # dimensi embedding dokumen
EPOCHS_D2V = 40

d2v_model = Doc2Vec(
    vector_size=VECTOR_SIZE,
    window=5,
    min_count=2,
    workers=4,
    dm=1,        # 1 = Distributed Memory (doc2vec DM), 0 = DBOW
    epochs=EPOCHS_D2V
)

d2v_model.build_vocab(train_tagged)
print("Jumlah kata di vocab:", len(d2v_model.wv))

d2v_model.train(
    train_tagged,
    total_examples=d2v_model.corpus_count,
    epochs=d2v_model.epochs
)

Jumlah kata di vocab: 3623


In [12]:
def get_doc_vectors(model, texts):
    vectors = []
    for text in texts:
        tokens = text.split()
        vec = model.infer_vector(tokens)
        vectors.append(vec)
    return np.array(vectors)

X_train_vec = get_doc_vectors(d2v_model, X_train)
X_val_vec   = get_doc_vectors(d2v_model, X_val)
X_test_vec  = get_doc_vectors(d2v_model, X_test)

print("Shape X_train_vec:", X_train_vec.shape)
print("Shape X_val_vec  :", X_val_vec.shape)
print("Shape X_test_vec :", X_test_vec.shape)


Shape X_train_vec: (5024, 300)
Shape X_val_vec  : (1077, 300)
Shape X_test_vec : (1077, 300)


In [13]:
# ==================================================
# 7A. MODEL DENSE (MLP) UNTUK KLASIFIKASI
# ==================================================
model_dense = Sequential()
model_dense.add(Dense(128, activation="relu", input_shape=(VECTOR_SIZE,)))
model_dense.add(Dropout(0.5))
model_dense.add(Dense(64, activation="relu"))
model_dense.add(Dropout(0.5))
model_dense.add(Dense(1, activation="sigmoid"))

optimizer = optimizers.Adam(learning_rate=1e-3)

model_dense.compile(
    loss="binary_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"]
)

model_dense.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        38,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,849 (183.00 KB)

 Trainable params: 46,849 (183.00 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# ==================================================
# 8A. TRAINING MODEL DENSE
# ==================================================
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

BATCH_SIZE = 64
EPOCHS = 30

history_dense = model_dense.fit(
    X_train_vec, y_train,
    validation_data=(X_val_vec, y_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.5964 - loss: 0.6665 - val_accuracy: 0.7456 - val_loss: 0.5523
Epoch 2/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7285 - loss: 0.5609 - val_accuracy: 0.7512 - val_loss: 0.5341
Epoch 3/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7464 - loss: 0.5390 - val_accuracy: 0.7623 - val_loss: 0.5223
Epoch 4/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7532 - loss: 0.5261 - val_accuracy: 0.7651 - val_loss: 0.5161
Epoch 5/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7673 - loss: 0.5194 - val_accuracy: 0.7595 - val_loss: 0.5225
Epoch 6/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7690 - loss: 0.4998 - val_accuracy: 0.7688 - val_loss: 0.5057
Epoch 7/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7683 - loss: 0.4949 - val_accuracy: 0.7595 - val_loss: 0.5100
Epoch 8/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7892 - loss: 0.4657 - val_accuracy: 0.7725 - val_loss

In [15]:
# ==================================================
# 9A. EVALUASI TEST (DENSE)
# ==================================================
y_pred_prob_dense = model_dense.predict(X_test_vec)
y_pred_dense = (y_pred_prob_dense >= 0.5).astype(int)

print("=== Classification Report (Test, Dense) ===")
print(classification_report(y_test, y_pred_dense, target_names=["Non-sarkasme", "Sarkasme"]))

print("=== Confusion Matrix (Test, Dense) ===")
print(confusion_matrix(y_test, y_pred_dense))


34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
=== Classification Report (Test, Dense) ===
              precision    recall  f1-score   support

Non-sarkasme       0.79      0.82      0.80       657
    Sarkasme       0.69      0.65      0.67       420

    accuracy                           0.75      1077
   macro avg       0.74      0.74      0.74      1077
weighted avg       0.75      0.75      0.75      1077

=== Confusion Matrix (Test, Dense) ===
[[536 121]
 [145 275]]


In [20]:
def clean_text_for_prediction(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text) # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces
    return text

def predict_sarcasm(text, model, d2v_model):
    text_clean = clean_text_for_prediction(text)
    vec = d2v_model.infer_vector(text_clean.split())
    vec = np.expand_dims(vec, axis=0)  # (1, VECTOR_SIZE)
    prob = model.predict(vec)[0][0]
    label = 1 if prob >= 0.5 else 0
    return label, prob

contoh = "Wah hebat banget, kerja segitu aja udah capek."
label, prob = predict_sarcasm(contoh, model_dense, d2v_model)
print("Probabilitas sarkasme:", prob)
print("Label:", "Sarkasme" if label == 1 else "Non-sarkasme")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Probabilitas sarkasme: 0.14970687
Label: Non-sarkasme
